## **Leaf Counter**

In [ ]:
if __name__ == "__main__":
    # For a single image
    """
    image_path = "images/leaf count/new folder/f.jpg"  # Replace with your image path
    count_leaves(image_path)
    """
    image_paths = []
    input_path = Path('images/leaf count/new folder')
    for image_file in input_path.rglob('*.jpg'):
        image_paths.append(image_file)
    
    leaf_counts = []
    for image_path in image_paths:
        leaf_count, _ = count_leaves(image_path, display_results=True)
        leaf_counts.append(leaf_count)
    
    # Plotting leaf count growth over time
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, len(leaf_counts) + 1), leaf_counts, marker='o', linestyle='-', color='g')
    plt.title('Leaf Growth Analysis Over Time')
    plt.xlabel('Image Number')
    plt.ylabel('Leaf Count')
    plt.xticks(range(1, len(leaf_counts) + 1))
    plt.grid()
    plt.show()
    #"""

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

def count_leaves(image_path, display_results=True, min_contour_area=50):
    """
    Count leaves in a seedling image using watershed segmentation.
    
    Parameters:
    - image_path: Path to the image file
    - display_results: Whether to display visualization of results
    - min_contour_area: Minimum contour area to be considered a leaf
    
    Returns:
    - leaf_count: Number of leaves detected
    - result_image: Image with leaf annotations
    """
    # Load image
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Error loading image: {image_path}")
        return 0, None
    
    # Create a copy for results visualization
    result_image = image.copy()
    
    # Resize for consistency
    image = cv2.resize(image, (600, 600))
    result_image = cv2.resize(result_image, (600, 600))
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    
    # Convert to HSV color space
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    
    # Define multiple green color ranges to catch different shades
    lower_green1 = np.array([25, 40, 40])
    upper_green1 = np.array([95, 255, 255])
    mask1 = cv2.inRange(hsv, lower_green1, upper_green1)
    
    lower_green2 = np.array([30, 30, 30])
    upper_green2 = np.array([90, 255, 255])
    mask2 = cv2.inRange(hsv, lower_green2, upper_green2)
    
    # Combine masks
    mask = cv2.bitwise_or(mask1, mask2)
    
    # Define kernels for morphological operations
    small_kernel = np.ones((3, 3), np.uint8)
    medium_kernel = np.ones((5, 5), np.uint8)
    
    # Apply morphological operations
    # Opening to remove noise
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, small_kernel, iterations=3)
    
    # Dilate to expand leaf regions
    dilated_mask = cv2.dilate(mask, medium_kernel, iterations=3)
    
    # Find the largest contour (likely the main plant)
    contours, _ = cv2.findContours(dilated_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        print("No plant detected in the image")
        return 0, result_image
    
    # Get the largest contour (main plant)
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Create a mask of just the main plant region
    plant_mask = np.zeros_like(mask)
    cv2.drawContours(plant_mask, [largest_contour], -1, 255, -1)
    
    # Apply the plant mask to the original mask
    mask = cv2.bitwise_and(mask, plant_mask)
    
    # Use watershed algorithm to separate touching leaves
    # Background area
    sure_bg = cv2.dilate(mask, medium_kernel, iterations=3)
    
    # Finding sure foreground area using distance transform
    dist_transform = cv2.distanceTransform(mask, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.3*dist_transform.max(), 255, 0)  # Reduced threshold to better separate
    sure_fg = np.uint8(sure_fg)
    
    # Finding unknown region
    unknown = cv2.subtract(sure_bg, sure_fg)
    
    # Marker labeling
    ret, markers = cv2.connectedComponents(sure_fg)
    
    # Add one to all labels so that background is not 0, but 1
    markers = markers + 1
    
    # Mark the unknown region with 0
    markers[unknown == 255] = 0
    
    # Apply watershed algorithm
    markers = cv2.watershed(image, markers)
    
    # Count watershed segments (excluding background and boundaries)
    # Background = 1, boundaries = 0, actual segments > 1
    unique_markers = np.unique(markers)
    watershed_segments = len([m for m in unique_markers if m > 1])
    
    # Create a mask where each watershed region gets a unique color
    leaf_mask = np.zeros_like(mask)
    leaf_mask[markers > 1] = 255
    
    # Find contours in the watershed result
    contours, _ = cv2.findContours(leaf_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter contours based on area
    valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
    
    # Use watershed segment count if it seems reasonable
    if 0 < watershed_segments <= 10:  # Sanity check: most seedlings won't have more than 10 leaves
        leaf_count = watershed_segments
    else:
        leaf_count = len(valid_contours)
    
    # Draw contours and add annotations
    for i, cnt in enumerate(valid_contours):
        # Get bounding box
        x, y, w, h = cv2.boundingRect(cnt)
        
        # Draw rectangle around leaf
        cv2.rectangle(result_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Calculate centroid for labeling
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            # Label each leaf with a number
            cv2.putText(result_image, str(i+1), (cx, cy), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    
    # Add leaf count text to the result image
    cv2.putText(result_image, f'Leaves: {leaf_count}', (20, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    if display_results:
        # Create a figure with subplots for visualization
        plt.figure(figsize=(15, 10))
        
        # Original image
        plt.subplot(2, 2, 1)
        plt.title("Original Image")
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        
        # Mask visualization
        plt.subplot(2, 2, 2)
        plt.title("Plant Mask")
        plt.imshow(mask, cmap='gray')
        plt.axis("off")
        
        # Watershed result
        plt.subplot(2, 2, 3)
        plt.title(f"Watershed Segmentation: {watershed_segments} segments")
        watershed_display = np.zeros((markers.shape[0], markers.shape[1], 3), dtype=np.uint8)
        for i in range(2, markers.max() + 1):
            watershed_display[markers == i] = [np.random.randint(0, 255) for _ in range(3)]
        plt.imshow(watershed_display)
        plt.axis("off")
        
        # Final result with leaf count
        plt.subplot(2, 2, 4)
        plt.title(f"Leaf Count: {leaf_count}")
        plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        
        plt.tight_layout()
        plt.show()
    
    print(f'Watershed segments: {watershed_segments}')
    print(f'Total Leaves Counted: {leaf_count}')
    return leaf_count, result_image

# Example usage
if __name__ == "__main__":
    # For a single image
    """
    image_path = "image.jpg"  # Replace with your image path
    count_leaves(image_path)
    
    # For multiple images in a directory
    """
    image_paths = []
    input_path = Path('images/leaf count/new folder')
    for image_file in input_path.rglob('*.jpg'):
        image_paths.append(image_file)
    
    leaf_counts = []
    for image_path in image_paths:
        leaf_count, _ = count_leaves(image_path, display_results=True)
        leaf_counts.append(leaf_count)
    
    # Plotting leaf count growth over time
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, len(leaf_counts) + 1), leaf_counts, marker='o', linestyle='-', color='g')
    plt.title('Leaf Growth Analysis Over Time')
    plt.xlabel('Image Number')
    plt.ylabel('Leaf Count')
    plt.xticks(range(1, len(leaf_counts) + 1))
    plt.grid()
    plt.show()